In [1]:
import os
import pandas as pd
import requests
import gzip

**Generate human chromosome size data**


In [ ]:
def fetch_chrom_sizes(genome, output_file):
    """Utility for downloading chromosome sizes from UCSC."""
    full_file_path = os.path.join(os.getcwd(), output_file)

    url = f"http://hgdownload.soe.ucsc.edu/goldenPath/{genome}/bigZips/{genome}.chrom.sizes"
    response = requests.get(url)

    if response.status_code == 200:
        with open(full_file_path, "w") as f:
            for line in response.text.splitlines():
                chrom, size = line.split()
                f.write(f"{chrom}\t{size}\n")
        print(f"Chromosome sizes saved to {full_file_path}")
    else:
        print(f"Failed to fetch chromosome sizes for {genome}")


output_file = "chromosome_sizes.txt"
fetch_chrom_sizes("hg38", output_file)

Chromosome sizes saved to /Users/siyuanzhao/Documents/GitHub/CS522_Project/Scripts/chromosome_sizes.txt


**Generate folding input files**


In [6]:
import os
import pandas as pd

def make_dir(d):
    """Utility for making a directory if not existing."""
    if not os.path.exists(d):
        os.makedirs(d)

def get_spe_inter(hic_data, alpha=0.05):
    """Filter Hi-C data for significant interactions based on the alpha threshold."""
    hic_spe = hic_data.loc[hic_data['fdr'] < alpha]
    return hic_spe

def get_fold_inputs(spe_df):
    """Prepare folding input file from the filtered significant interactions."""
    spe_out_df = spe_df[['ibp', 'jbp', 'fq', 'chr', 'fdr']]
    spe_out_df['w'] = 1
    result = spe_out_df[['chr', 'ibp', 'jbp', 'fq', 'w']]
    return result

def process_hic_files(input_folder, seqs_folder, output_folder, alpha=0.05):
    """Process Hi-C files by matching with seqs files for reference data and save results in the output folder."""
    
    make_dir(output_folder)
    
    # Iterate through each file in the Hi-C input folder
    for hic_file_name in os.listdir(input_folder):
        if hic_file_name.endswith(".csv.gz"):
            key_word = hic_file_name.split("_")[0]  # Assume keyword is the prefix before the first underscore
            
            # Find the matching reference file in the seqs folder
            seq_file_name = f"{key_word}_ranges.csv.gz"
            seq_file_path = os.path.join(seqs_folder, seq_file_name)
            
            if not os.path.exists(seq_file_path):
                print(f"No matching reference file found for {hic_file_name}")
                continue
            
            # Load Hi-C data and the corresponding reference data
            hic_file_path = os.path.join(input_folder, hic_file_name)
            all_hic = pd.read_csv(hic_file_path)
            spe_hic = get_spe_inter(all_hic, alpha)
            reference_df = pd.read_csv(seq_file_path, usecols=["chrID", "start_value", "end_value"])

            # Process each row in the reference file for filtering
            for _, row in reference_df.iterrows():
                chrID = row["chrID"]
                start_value = row["start_value"]
                end_value = row["end_value"]
                
                # Filter Hi-C data based on chrID and ibp range
                chr_hic_data = spe_hic[
                    (spe_hic["chr"] == chrID) &
                    (spe_hic["ibp"] >= start_value) &
                    (spe_hic["ibp"] <= end_value)
                ]
                
                if chr_hic_data.empty:
                    continue

                # Prepare folding input data
                fold_hic = get_fold_inputs(chr_hic_data)

                # Generate output file name and save
                output_file_name = f"{key_word}.{chrID}.{start_value}.{end_value}.txt"
                fold_hic_path = os.path.join(output_folder, output_file_name)

                fold_hic.to_csv(fold_hic_path, header=None, index=None, sep="\t", mode="a")

input_folder = '../Data/refined_processed_HiC'
seqs_folder = '../Data/seqs'
output_folder = '../Data/Folding_input'
process_hic_files(input_folder, seqs_folder, output_folder)


**Epigenetic tracks**

In [ ]:
with gzip.open("../Example_Data/epigenetic_tracks/GM_H3K27ac.bed.gz", "rt") as f:
    df = pd.read_csv(f, sep="\t", header=None)

    df.columns = ["chrom", "start", "end", "name", "score", "strand", "signalValue", "pValue", "qValue", "peak"]


df.head()
df.tail()


,chrom,start,end,name,score,strand,signalValue,pValue,qValue,peak
53684,chrX,9941902,9942167,Peak_44631,64,.,5.30866,6.46416,4.53197,202
53685,chrX,9942471,9942798,Peak_48539,59,.,4.94979,5.94877,4.03142,146
53686,chrX,997401,998613,Peak_37227,79,.,5.83975,7.99793,6.00050,816
53687,chrX,999210,999606,Peak_48540,59,.,4.50481,5.94877,4.03142,173
53688,chrX,9997445,9997647,Peak_42601,68,.,5.04212,6.80304,4.85152,95
